In [1]:
#JV

In [2]:
import numpy as np

import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
import random

import os
import gc
import time
import math

import matplotlib.pyplot as plt
#plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

from tqdm.notebook import tqdm

In [3]:
seed = 23

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
class LanguageProcessor:

    def __init__(self,language_directory,target_lang_name,mode="train",meta_tokens=True):

        """
        Default Constructor for this class.

        Params:

            language_directory : ex : "aksharantar_sampled/tel/"
            mode : "train" or "test" or "valid", accordingly the appropriate dataset is read.
            meta_tokens : If true creates the first three tokens of the dictionary as <start>,<end>,<pad>.
            
        """

        self.meta_tokens = meta_tokens ## boolean variable, if 1, then <start>,<end> and <pad> tokens are cosidered in the vocab.
        self.language_directory = language_directory
        self.target_lang_name = target_lang
        self.mode = mode ## accordingly helps to read and generate the appropriate dataset.
    
        self.source_lang = 0
        self.target_lang = 1

        self.source_max_len = self.find_max_len(self.source_lang)
        self.target_max_len = self.find_max_len(self.target_lang)

        self.max_len = max(self.source_max_len,self.target_max_len)+1 ##accomodating End token also, irrespective of whether it is used.

        self.source_char2id,self.source_id2char = self.build_char_vocab(self.source_lang,self.source_max_len)
        self.target_char2id,self.target_id2char = self.build_char_vocab(self.target_lang,self.target_max_len)


    def find_max_len(self,lang):

        """
        Method to find the maximum length of a word across train/test and validation data.

        This would help in padding, the embedding accordingly.

        Params:

            lang : 0/1 (source/target) language for which the length of the longest word must be found.
        
        """

        train_df = pd.read_csv(self.language_directory+self.target_lang_name+"_train.csv",header=None)
        test_df = pd.read_csv(self.language_directory+self.target_lang_name+"_test.csv",header=None)
        valid_df = pd.read_csv(self.language_directory+self.target_lang_name+"_valid.csv",header=None)

        train_max_len = len(max(list(train_df[lang]), key = len))
        test_max_len = len(max(list(test_df[lang]), key = len))
        valid_max_len = len(max(list(valid_df[lang]), key = len))

        del train_df
        del test_df
        del valid_df

        gc.collect()

        return max(train_max_len,test_max_len,valid_max_len)

    def build_char_vocab(self,lang_id,max_len=None):

        """
        Method to create a vocabulary of characters in language corresponding to lang_id.
        """

        df = pd.read_csv(self.language_directory+self.target_lang_name+"_"+self.mode+".csv",header=None)

        self.data = df.to_numpy()

        lang_chars = []
        lang_words = df[lang_id].to_numpy()
    
        for word in lang_words:
            lang_chars += list(word)
    
        unique_lang_chars =  sorted(list(set(lang_chars)))
        
        if self.meta_tokens:
            char2id_dict = {'<start>':0,'<end>':1,'<pad>': 2}
            id2char_dict = {0:'<start>',1:'<end>',2:'<pad>'}
            
        else:
            char2id_dict = {}
            id2char_dict = {}

        start = len(char2id_dict) ##Key of each language character starts based on meta tokens are used or not.
    
        for i in range(len(unique_lang_chars)):
            char2id_dict[unique_lang_chars[i]] = i+start
            id2char_dict[i+start] = unique_lang_chars[i]
    
        del df
        del lang_chars
        del unique_lang_chars

        gc.collect()
    
        return char2id_dict,id2char_dict

    def encode_word(self,word,lang_id,padding=True,append_eos = True):

        """
        Method to encode characters of a given word.

        Params:

            word: The word to be encoded.
            lang_id : 0/1 for source/target lang.
            padding : If true, the word encoding would be padded upto max len.
            append_eos : Appends <end> token at the end of every word.
        
        """

        if lang_id == self.source_lang:
            char2id_dict = self.source_char2id
            
        else:
            char2id_dict = self.target_char2id
        
        max_len = self.max_len

        word_encoding = []
        
        #if lang_id == self.source_lang:
        #    word_encoding = [char2id_dict['<start>']] ##every input starts with the <start> token.
        
        for i in word.lower():
            word_encoding.append(char2id_dict[i])

        #if append_eos:
        #    word_encoding.append(char2id_dict['<end>'])

        ## pad till maxlen, if padding is used.
        if padding:
            word_encoding += [char2id_dict['<pad>']] * (max_len - len(word_encoding))
        
        return np.array(word_encoding)

    def decode_word(self,code_word,lang_id):

        """
        Method to decode an encoded word.

        Params:

            code_word : The encoded word.
            lang_id : 0/1 for source/target lang.
        """
    
        word = []

        if lang_id == self.source_lang:
            id2char_dict = self.source_id2char
            char2id_dict = self.source_char2id
            
        else:
            id2char_dict = self.target_id2char
            char2id_dict = self.target_char2id

        start_idx = 0#1-lang_id
        
        for i in code_word[start_idx:]:
            ## if we reached <end>, then stop decoding
            if self.meta_tokens and i == char2id_dict['<end>'] or i == char2id_dict['<pad>']:
                break
            
            word.append(id2char_dict[i])
            
        return np.array(word)
            

In [5]:
class WordDataset(Dataset):

    """
    Class that inherits and overrides the methods of Dataset class. This helps in creating a data loader.
    """
    
    def __init__(self, language_processor,append_eos=True,device='cpu'):

        self.lp = language_processor
        self.train_lp = train_lp
        self.data = self.lp.data
        self.device = device
        self.append_eos = append_eos

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_word, output_word = self.data[idx]
        
        input_sequence = self.lp.encode_word(input_word,self.lp.source_lang,padding=False,append_eos=self.append_eos)
        output_sequence = self.lp.encode_word(output_word,self.lp.target_lang,padding=False,append_eos=self.append_eos)
        

        #if len(input_sequence) != len(output_sequence):
        #    print(input_word,len(input_word),output_word,len(output_word))
        
        return torch.tensor(input_sequence).to(device), torch.tensor(output_sequence).to(device)
        #return input_sequence, output_sequence


In [6]:
def collate_fn(batch):
    
    input_words, target_words = zip(*batch)
    
    padded_inputs = pad_sequence(input_words, batch_first=True, padding_value=pad_token_id)
    
    padded_targets = pad_sequence(target_words, batch_first=True, padding_value=pad_token_id)
    
    input_lengths = torch.LongTensor([len(seq) for seq in input_words]).to(device)
    target_lengths = torch.LongTensor([len(seq) for seq in target_words]).to(device)
    
    return padded_inputs, padded_targets, input_lengths, target_lengths

In [7]:
batch_size = 64

base_dir = "aksharantar_sampled/"
target_lang = "tel"

use_meta_tokens = True
append_eos = 1

lang_dir = base_dir + target_lang + "/"

In [8]:
device = torch.device("mps")

##creating train loader
train_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="train",meta_tokens=use_meta_tokens)
train_dataset = WordDataset(train_lp,device=device)
train_loader = DataLoader(train_dataset, batch_size=batch_size,collate_fn=collate_fn, shuffle=True)

## creating test loader
test_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="test",meta_tokens=use_meta_tokens)
## to make sure that the same vocabulary and dictionaries are used everywhere
test_lp.source_char2id = train_lp.source_char2id
test_lp.source_id2char = train_lp.source_id2char
test_lp.target_char2id = train_lp.target_char2id
test_lp.target_id2char = train_lp.target_id2char

test_dataset = WordDataset(test_lp,device=device)
test_loader = DataLoader(test_dataset, batch_size=batch_size,collate_fn=collate_fn, shuffle=True)

## creating validation loader
valid_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="valid",meta_tokens=use_meta_tokens)
valid_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="valid",meta_tokens=use_meta_tokens)
valid_lp.source_char2id = train_lp.source_char2id
valid_lp.source_id2char = train_lp.source_id2char
valid_lp.target_char2id = train_lp.target_char2id
valid_lp.target_id2char = train_lp.target_id2char
valid_dataset = WordDataset(valid_lp,device=device)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size,collate_fn=collate_fn, shuffle=True)

In [9]:
##in principle these are all fixed across train/test/valid data

## the ids of these tokens are the same in the source and target language
start_token_id = train_lp.source_char2id['<start>']
end_token_id = train_lp.source_char2id['<end>']
pad_token_id = train_lp.source_char2id['<pad>']

SOS_token = start_token_id

#source_max_len = train_lp.source_max_len
#target_max_len = train_lp.target_max_len

In [10]:
## Verifying if the dataloaders are carrying data correctly.
count = 0
for data in valid_loader:

    inp,tar,_,_ = data

    inp = inp.view(batch_size,-1).cpu().numpy()
    tar = tar.view(batch_size,-1).cpu().numpy()

    for i in range(min(batch_size,32)):
        print("".join(list(train_lp.decode_word(inp[i],0))),"".join(list(train_lp.decode_word(tar[i],1))))



prajektullo ప్రాజెక్టుల్లో
nirviryam నిర్వీర్యం
thalakrimdulugaa తలక్రిందులుగా
sadhinchina సాధించినా
slaat స్లాట్
poralato పొరలతో
kaaryaaniki కార్యానికి
saantaa శాంతా
kolambonu కొలంబోను
praamthamulu ప్రాంతములు
saahiti సాహితి
ambareeshurudu అంబరీషుడు
gunavatiki గుణవతికి
intilooni ఇంటిలోని
undalekapoyaadu ఉండలేకపోయాడు
sandhaanam సంధానం
anugrahinchi అనుగ్రహించి
grandhamaala గ్రంథమాల
aadamtho ఆడంతో
dhis థిస్
pooshaaru పోశారు
phoor ఫ్లోర్
pranatyagam ప్రాణత్యాగం
uttaraayanam ఉత్తరాయణం
bhaareelakshamto భారీలక్షంతో
kondalla కొండల్లా
sodanu సోడాను
kreedaakaarulandariki క్రీడాకారులందరికి
edlanu ఎడ్లను
eerooje ఈరోజే
phool ఫూల్
raghuveer రఘువీర్
parachaadu పరచాడు
karol కారల్
bashan బాషన్
churchini చర్చిని
yimpu యింపు
aayurvedamu ఆయుర్వేదము
granthamala గ్రంథమాల
vetta వేత్త
kaartheeka కార్తీక
rashmi రష్మి
swaraparachanunnamani స్వరపరచనున్నామని
paali పాలి
tavvagaa తవ్వగా
canadalo కెనడాలో
padu పాడు
chaerukunnadi చేరుకున్నది
chesthunnadhi చేస్తున్నది
mantramunaku మంత్రమునకు
naukaree నౌకరీ
spandinchata

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1,num_layers=1,bidirectional=False):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size,num_layers = num_layers,bidirectional=bidirectional, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [12]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size,D,expected_dim,batch_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size*expected_dim, hidden_size)
        self.Ua = nn.Linear(hidden_size*D, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)
        self.batch_size = batch_size

    def forward(self, query, keys):

        #print(f"(encoder_outputs)keys.shape:{keys.shape}")
        #print(f"\nquery.shape:{query.shape}")
        reshaped_query = query.reshape(self.batch_size,1,-1)
        #print(f"reshaped_query.shape:{reshaped_query.shape}")
        #print(f"Wa(query.reshape(batch_size,1,-1)).shape : {self.Wa(reshaped_query).shape}")
        #print(f"Ua(keys).shape : {self.Ua(keys).shape}")
        
        scores = self.Va(torch.tanh(self.Wa(query.reshape(batch_size,1,-1)) + self.Ua(keys)))

        #print(f"scores.shape : {scores.shape}")
        scores = scores.squeeze(2).unsqueeze(1)
        #print(f"scores.squeeze(2).unsqueeze(1).shape : {scores.shape}")

        weights = F.softmax(scores, dim=-1)
        #print(f"weights.shape : {weights.shape}\n")
        context = torch.bmm(weights, keys)

        return context, weights

In [13]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,batch_size, dropout_p=0.1,num_layers=1,bidirectional=False):
        super(AttnDecoderRNN, self).__init__()

        self.num_layers = num_layers
        self.D = 1 ##the number of directions in which the input is viewed.
        if bidirectional:
            self.D = 2
            
        ## In h0 (the input to the decoder) first dimension expected is number of directions X number of layers 
        self.expected_h0_dim1 = self.D*self.num_layers
        
        self.embedding = nn.Embedding(output_size, hidden_size*self.D)
        self.attention = BahdanauAttention(hidden_size,self.D,self.expected_h0_dim1,batch_size)
        self.gru = nn.GRU(2 * self.D * hidden_size, hidden_size,num_layers=self.num_layers,bidirectional = bidirectional, batch_first=True)
        self.out = nn.Linear(hidden_size*self.D, output_size)
        self.dropout = nn.Dropout(dropout_p)
        

        #print(f"expected_h0_dim1:{self.expected_h0_dim1}\n\n")

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None,MAX_LENGTH=28):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))
        

        if hidden.shape[0] != self.expected_h0_dim1:
            reshaped_hidden = hidden.repeat(self.expected_h0_dim1,1,1)
        else:
            reshaped_hidden = hidden

        #print(f"encoder_outputs.shape:{encoder_outputs.shape}")
        #print(f"hidden.shape:{hidden.shape}")
        #print(f"reshaped_hidden.shape:{reshaped_hidden.shape}")

        query = reshaped_hidden.permute(1, 0, 2)
        #print(f"query.shape:{query.shape}")
        context, attn_weights = self.attention(query, encoder_outputs)
        #print(f"embedded.shape:{embedded.shape}")
        #print(f"context.shape:{context.shape}")
        #print(f"attn_weights.shape:{attn_weights.shape}")
        input_gru = torch.cat((embedded, context), dim=2)
        #print(f"input_gru.shape:{input_gru.shape}")
        
        

        output, hidden = self.gru(input_gru, reshaped_hidden)
        #print(f"output1.shape:{output.shape}")
        output = self.out(output)
        #print(f"output2.shape:{output.shape}")
        #print("========================================================================")

        return output, hidden, attn_weights

In [14]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion,ignore_padding=True):

    tot_correct_word_preds = 0
    tot_words = 0
    
    total_loss = 0
    for data in tqdm(dataloader):
        input_tensor, target_tensor,_,tar_max_len = data

        max_len = torch.max(tar_max_len).item()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)

        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor,max_len)

        multi_step_preds = torch.argmax(decoder_outputs,dim=2)
        multi_step_pred_correctness = (multi_step_preds ==  target_tensor)
        num_words = multi_step_preds.shape[0]
        
        if ignore_padding: ## if padding has to be ignored.

            ## for each word, based on the padding token ID, find the first occurance of the padding token, marking the begining of padding.
            
            ## argmax is not supported for bool on cuda, hence casting it to long.
            padding_start = torch.argmax((target_tensor == pad_token_id).to(torch.long),dim=1).to(device)
            ## Creating a mask with 1's in each position of a padding token
            mask = (torch.arange(target_tensor.size(1)).unsqueeze(0).to(device) >= padding_start.unsqueeze(1))
            #print(mask)
            
            ##doing a logical OR with the mask makes sure that the padding tokens do not affect the correctness of the word
            tot_correct_word_preds += (torch.all(torch.logical_or(multi_step_pred_correctness,mask),dim=1).int().sum()).item()
            tot_words += num_words

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    epoch_loss = round(total_loss / len(dataloader),4)
    epoch_accuracy = round(tot_correct_word_preds*100/tot_words,2)

    return epoch_loss,epoch_accuracy



In [15]:
def train(train_dataloader,valid_dataloader, encoder, decoder,loss_criterion, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    for epoch in tqdm(range(1, n_epochs + 1)):
        train_loss,train_accuracy = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_criterion)
        print_loss_total += train_loss
        plot_loss_total += train_loss
        val_loss,_,val_word_level_accuracy = compute_accuracy(valid_loader,encoder,decoder,loss_criterion,padding_token_id = train_loader.dataset.train_lp.source_char2id['<pad>'],end_token_id = train_loader.dataset.train_lp.source_char2id['<end>'],ignore_padding=True,device=device)

        print(f"Epoch {epoch}\t Train Loss : {train_loss}\t Train Acc : {train_accuracy}% \t Val Loss : {val_loss}\t Val Acc : {val_word_level_accuracy}%")
        
        #print(f"JV Char Accuracy:{JV_char_accuracy}%\t JV Word Accuracy:{JV_word_accuracy}%\t Loss : {loss}")

In [16]:
def compute_accuracy(dataloader,encoder,decoder,loss_criterion,padding_token_id,end_token_id = 1,ignore_padding = True,device='cpu'):

    """
    Method to compute the accuracy using the model (encoder-decoder) using dataloader.

    This method returns word and character level accuracy.

        Word Level Accuracy : Accuracy is computed at the word level and a word is right iff every character is predicted correctly.
        Char Level Accuracy : Accuracy is computed by comparing each predicted character wrt the correct char.

    Params:

        dataloader : The train/test/valid dataloader.
        encoder : The encoder 
        decoder : The decoder
        padding_token_id : The id of the padding token.
        ignore_padding : If True, then in word level accuracy, the padding characters are ignored in computing the word level accuracy.
                        char level accuracy, the padding characters are not considered at all.

                        If false, padding is considered to be a part of the word (for word level accuracy) and 
    """

    word_level_accuracy = 0

    tot_words = 0

    tot_correct_word_preds = 0

    loss = 0

    criterion = loss_criterion.to(device)

    with torch.no_grad():

        train = 0

        if encoder.training and decoder.training: ## reset the the model back to train mode
            train = 1

        encoder.eval()
        decoder.eval()

        for data in dataloader:
            
            input_tensor, target_tensor,_,tar_max_len = data

            max_len = torch.max(tar_max_len).item()
    
            encoder_outputs, encoder_hidden = encoder(input_tensor)
            ## even though we are passing target tensor, the teacher forcing ratio is 0, so no teacher forcing
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor,max_len)

            loss += criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_tensor.view(-1)).item()
    
            ## For a batch, for each character find the most probable output word.
            multi_step_preds = torch.argmax(decoder_outputs,dim=2)
            multi_step_pred_correctness = (multi_step_preds ==  target_tensor)
            num_chars = multi_step_preds.numel() ##find the total number of characters in the current batch
            num_words = multi_step_preds.shape[0] ##find the total number of words in the current batch.
    
            if ignore_padding: ## if padding has to be ignored.
    
                ## for each word, based on the padding token ID, find the first occurance of the padding token, marking the begining of padding.
                
                ## argmax is not supported for bool on cuda, hence casting it to long.
                padding_start = torch.argmax((target_tensor == pad_token_id).to(torch.long),dim=1).to(device)
                ## Creating a mask with 1's in each position of a padding token
                mask = (torch.arange(target_tensor.size(1)).unsqueeze(0).to(device) >= padding_start.unsqueeze(1))
                #print(mask)
                
                ##doing a logical OR with the mask makes sure that the padding tokens do not affect the correctness of the word
                tot_correct_word_preds += (torch.all(torch.logical_or(multi_step_pred_correctness,mask),dim=1).int().sum()).item()
                tot_words += num_words
    
            else: ##otherwise.
    
                tot_correct_word_preds += (torch.all(multi_step_pred_correctness,dim=1).int().sum()).item()
                tot_words += num_words
                
                tot_correct_char_preds += (multi_step_pred_correctness.int().sum()).item()
                tot_chars += num_chars

        #print(tot_correct_char_preds,tot_chars)
        #print(tot_correct_word_preds,tot_words)
    
        word_lvl_accuracy = round(tot_correct_word_preds*100/tot_words,2)

        loss /= dataloader.dataset.data.shape[0]

        if train:

            encoder.train()
            decoder.train()
    
        return round(loss,4),None,word_lvl_accuracy

In [17]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
hidden_size = 128
#batch_size = 32

input_vocab_size = len(train_lp.source_char2id)
output_vocab_size = len(train_lp.target_char2id)

loss_criterion = nn.CrossEntropyLoss(ignore_index=pad_token_id)

encoder = EncoderRNN(input_vocab_size, hidden_size,num_layers=2,bidirectional=True).to(device)
decoder = AttnDecoderRNN(hidden_size, output_vocab_size,batch_size = batch_size,num_layers=2,bidirectional=True).to(device)

train(train_loader,valid_loader, encoder, decoder,loss_criterion, 3, print_every=1, plot_every=5)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/800 [00:00<?, ?it/s]

#### To Do

1. Add support for multi-layers [currently bidirectional is only supported]
2. Ignore Padding Index
3. Add support for LSTM and RNN
4. Include Teacher forcing, with ratio.
5. Add support for eval mode : to predict for individual words